In [65]:
#Imports
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import sklearn

from sklearn.model_selection import train_test_split 
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict

from sklearn.ensemble import RandomForestClassifier 
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import ExtraTreesClassifier


from sklearn import metrics 


In [66]:
#Load data
path = os.getcwd() + '/data/AllSites.csv'
ds1 = pd.read_csv(path)

#Remove rows with missing cyanobacteria values
ds1 = ds1.dropna(axis=0, subset = ['NP_Cya_bio'])

#Remove rows with any missing value? 
#Turns out that sklearn can't handle missing values so lets delete those rows for now. 
#Later we can fill them in with monthly means.
ds1 = ds1.dropna(axis=0, how='any')

#Add target column - I'll do this in the clean set.
#ds1['target'] = [1 if x >= 4e8 else 0 for x in ds1['NP_Cya_bio']]

print(ds1.shape)
print(type(ds1))
#print(ds1[ds1['target'] == 1])
ds1

(1298, 14)
<class 'pandas.core.frame.DataFrame'>


,StationID,Station,Date,Time,Stratum,Depth,TP,DP,Cl,TN,TempC,Chla,Secchi,NP_Cya_bio
1,2,South Lake B,05/02/06,1050.0,U,2.2,36.8,14.8,17.5,0.45,13.7,9.67,1.1,0.0
2,2,South Lake B,05/25/06,1100.0,U,2,50.1,27.4,12.1,0.55,14.5,2.04,0.7,0.0
4,2,South Lake B,06/08/06,1105.0,U,2,59.6,32.6,12,0.65,17.7,4.13,0.6,0.0
5,2,South Lake B,06/30/06,1050.0,U,1.2,77.3,47.9,10.5,0.62,22.5,1.74,0.6,0.0
7,2,South Lake B,07/24/06,1110.0,U,2,107,49.9,10.6,0.87,21.4,2.17,0.2,741000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3629,51,Missisquoi Bay Central,08/19/15,1104.0,U,2.2,53.4,16.8,8,0.69,25.6,27.5,1.1,389000000.0
3631,51,Missisquoi Bay Central,09/04/15,1117.0,U,2,83.4,33.9,8.3,0.71,23.7,23.94,1,133000000.0
3632,51,Missisquoi Bay Central,09/17/15,1100.0,U,2,94.2,40.7,8.7,0.9,22.3,50.16,1,443000000.0
3634,51,Missisquoi Bay Central,10/07/15,1042.0,U,2.8,68.8,42.6,9.6,0.74,13.4,10.22,1.4,9460000.0


In [67]:
#Create cleaned dataframe, ds2:
ds2 = ds1.drop(['StationID', 'Station', 'Date', 'Time', 'Stratum', 'Depth'], axis=1)

#Use regex to remove 'H's and 'J's
ds2['TP'] = ds1['TP'].astype(str).str.extract('([-+]?\d*\.\d+|\d+)').astype(float)
ds2['DP'] = ds1['DP'].astype(str).str.extract('([-+]?\d*\.\d+|\d+)').astype(float)
ds2['Cl'] = ds1['Cl'].astype(str).str.extract('([-+]?\d*\.\d+|\d+)').astype(float)
ds2['TN'] = ds1['TN'].astype(str).str.extract('([-+]?\d*\.\d+|\d+)').astype(float)
ds2['TempC'] = ds1['TempC'].astype(str).str.extract('([-+]?\d*\.\d+|\d+)').astype(float)
ds2['Chla'] = ds1['Chla'].astype(str).str.extract('([-+]?\d*\.\d+|\d+)').astype(float)
ds2['Secchi'] = ds1['Secchi'].astype(str).str.extract('([-+]?\d*\.\d+|\d+)').astype(float)
ds2['Month'] = ds1['Date'].astype(str).str.extract('(\d\d)').astype(int) # This is just the month number
ds2['N:P'] = ((ds2['TN']*1e-3)/14.007)/((ds2['TP']*1e-6)/30.974)
ds2['Target'] = [1 if x >= 4e8 else 0 for x in ds2['NP_Cya_bio']]

#Take a look
ds2
ds2[ds2['Target'] == 1]

,TP,DP,Cl,TN,TempC,Chla,Secchi,NP_Cya_bio,Month,N:P,Target
936,15.7,9.3,12.6,0.35,22.1,10.40,2.3,4.260000e+08,8,49.297008,1
3037,37.0,12.1,10.6,0.74,24.7,28.20,1.1,7.110000e+08,8,44.226458,1
3038,41.7,13.5,10.9,0.61,20.5,25.38,1.3,7.280000e+08,8,32.347889,1
3269,75.5,23.2,6.7,0.64,17.3,36.00,1.1,4.230000e+08,9,18.744989,1
3271,69.4,33.6,7.3,0.63,13.0,12.60,1.3,7.050000e+08,10,20.073969,1
3307,66.4,16.6,5.9,0.74,21.3,44.90,1.0,1.260000e+09,8,24.644261,1
3400,40.2,16.0,7.7,0.80,23.8,45.03,1.5,1.170000e+09,7,44.006426,1
3461,62.9,18.0,6.3,0.77,19.3,3.73,1.2,4.380000e+08,9,27.070249,1
3464,73.7,19.5,6.6,0.80,17.3,42.40,0.8,1.540000e+09,9,24.003505,1
3499,74.1,29.7,7.1,0.81,23.2,45.50,1.1,1.190000e+09,8,24.172356,1


In [68]:
#Create X and y
X = np.array(ds2.drop(['Target', 'NP_Cya_bio'], axis=1))
y = np.array(ds2['Target'])
y_reg = np.array(ds2['NP_Cya_bio']) #for regression

In [69]:
#Split the data - I think in this case since we're not really tuning any hyperparams it's ok to use cv for testing.
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, stratify = y)

In [70]:
#Random forest!! (Should I scale the data? No, not for trees or forests or PCA.)
trees = 500

model = RandomForestClassifier(n_estimators = trees, max_features = 'sqrt', criterion = 'entropy', class_weight = 'balanced')
#model.fit(X_train, y_train)
#y_pred = model.predict(X_test)

y_pred = cross_val_predict(model, X, y, cv=5)

scores = metrics.classification_report(y, y_pred)
confusion_matrix = metrics.confusion_matrix(y, y_pred)
print(scores)
print(confusion_matrix)

#Feature importance:
model.fit(X,y)
feature_importances = model.feature_importances_
print(feature_importances)

              precision    recall  f1-score   support

           0       0.99      0.97      0.98      1281
           1       0.17      0.41      0.25        17

    accuracy                           0.97      1298
   macro avg       0.58      0.69      0.61      1298
weighted avg       0.98      0.97      0.97      1298

[[1248   33]
 [  10    7]]
[0.12403801 0.03546606 0.10169647 0.22665467 0.03223341 0.26955295
 0.13164617 0.04328028 0.03543197]


In [71]:
#Just for fun, let's try ExtraTrees, too. 
model = ExtraTreesClassifier(n_estimators = trees, bootstrap = False, criterion = 'entropy', class_weight = 'balanced')
model.fit(X, y)
print(model.feature_importances_)

y_pred = cross_val_predict(model, X, y, cv=5)

scores = metrics.classification_report(y, y_pred)
confusion_matrix = metrics.confusion_matrix(y, y_pred)
print(scores)
print(confusion_matrix)

#Almost the same results. Boo.

[0.13186995 0.05543771 0.0898657  0.14481123 0.04936417 0.22780713
 0.1545078  0.09561355 0.05072276]
              precision    recall  f1-score   support

           0       0.99      0.97      0.98      1281
           1       0.17      0.41      0.24        17

    accuracy                           0.97      1298
   macro avg       0.58      0.69      0.61      1298
weighted avg       0.98      0.97      0.97      1298

[[1247   34]
 [  10    7]]


In [81]:
#Finally, let's try a random forest regression!
trees = 500

model = RandomForestRegressor(n_estimators = trees, max_features = 'auto', oob_score = True)
model.fit(X, y_reg)
print(model.feature_importances_)
model.oob_score_ #returns R^2 values using out of bag values as test sets

#Hmm. Doesn't seem great.

[0.04087562 0.04239199 0.05632923 0.04751955 0.03743043 0.66697676
 0.02413702 0.03233737 0.05200202]


0.5047993621767435

#### In Conclusion...
It doesn't seem like Random Forest is the best choice of model for this data set. Most unfortunately.